In [ ]:
from env import *
from agent import *
from metrics import compute_metrics, write_json
import copy
import torch
import pickle
import time
import numpy as np
from tqdm import tqdm
from scipy.stats import entropy
import argparse

from orion.client import report_results

try:
  import wandb 
  wandb_flag = True 
except:
  wandb_flag = False
  try:
      import mlflow
      mlflow_flag = True
  except:
      mlflow_flag = False

def init_logging(time_stamp, args):
    if wandb_flag:
        wandb.init(name=time_stamp, group="aarlc", project="medical_evidence_collection")
        wandb.config.update(args)
    elif mlflow_flag:
        mlflow.set_experiment(experiment_name=args.exp_name)
        mlflow.start_run()
        args = vars(args) if hasattr(args, "__dict__") else args
        for k in args:
            mlflow.log_param(k, args.get(k))
    else:
        pass



您提供的代码根据 wandb_flag 和 mlflow_flag 的值初始化日志记录。以下是对代码作用的解释： 如果 wandb_flag 为 True，则意味着将使用权重和偏差 (wandb) 库完成日志记录。在这种情况下，代码调用 wandb.init() 以使用指定的名称、组和项目初始化 wandb 运行。名称设置为 time_stamp 参数，组设置为“aarlc”，项目设置为“medical_evidence_collection”。初始化 wandb 后，代码使用 args 参数中的值更新 wandb 配置。 如果 wandb_flag 为 False 且 mlflow_flag 为 True，则意味着日志记录将使用 MLflow 库完成。在这种情况下，代码调用 mlflow.set_experiment() 将 MLflow 实验设置为 args.exp_name 的值。然后，它使用 mlflow.start_run() 启动新的 MLflow 运行。接下来，如果 args 具有 __dict__ 属性，代码使用 vars(args) 将 args 参数转换为字典，否则保持 args 不变。之后，它遍历 args 字典的键并使用 mlflow.log_param() 将每个键值对记录为参数。 如果 wandb_flag 和 mlflow_flag 都为 False，则表示没有启用日志库。在这种情况下，代码什么都不做。 总体而言，此代码旨在根据 wandb_flag 和 mlflow_flag 变量的值使用 wandb 或 MLflow 初始化和配置日志记录。如果两个标志都没有设置，则不执行日志记录。

In [ ]:
def log_metric_data(result):
    if wandb_flag:
        wandb.log(result)
    elif mlflow_flag:
        batch = result.pop("batch", 0) if "batch" in result else result.pop("epoch", 0)
        mlflow.log_metrics(result, batch)
    else:
        pass



您提供的代码用于根据 wandb_flag 和 mlflow_flag 的值记录指标数据。 以下是对代码作用的解释： 如果 wandb_flag 为 True，则意味着将使用 Weights & Biases (wandb) 库记录指标数据。 在这种情况下，代码使用结果参数调用 wandb.log()，该参数假定为包含要记录的度量数据的字典。 如果 wandb_flag 为 False 且 mlflow_flag 为 True，则意味着将使用 MLflow 库记录指标数据。 在这种情况下，代码会检查结果字典是否包含键“batch”。 如果是，它会从字典中删除键值对并将值分配给批处理变量。 如果“batch”键不存在，它会检查字典是否包含键“epoch”。 如果是，它会从字典中删除键值对并将值分配给批处理变量。 完成此步骤是为了提取批次或纪元信息以用于记录目的。 然后，代码使用结果字典和批处理值调用 mlflow.log_metrics() 以记录指标数据。 如果 wandb_flag 和 mlflow_flag 都为 False，则表示没有启用日志库。 在这种情况下，代码什么都不做。 总体而言，此代码旨在根据 wandb_flag 和 mlflow_flag 变量的值使用 wandb 或 MLflow 记录指标数据。 如果两个标志均未设置，则不执行日志记录。 该代码假定结果参数是一个包含度量数据的字典，并且如果日志记录需要，它会处理批次或纪元信息的提取。

In [ ]:
def main():
    print("Initializing Environment and generating Patients....")
    env = environment(args, args.train_data_path, train=True)
    print(f"Training environment size: {env.sample_size}")
    patience = args.patience
    best_val_accuracy = None
    if args.eval_on_train_epoch_end:
        eval_env = environment(args, args.val_data_path, train=False)
        print(f"Validation environment size: {eval_env.sample_size}")
    agent = Policy_Gradient_pair_model(state_size = env.state_size, disease_size = env.diag_size, symptom_size= env.symptom_size, LR = args.lr, Gamma = args.gamma)
    threshold_list = []
    best_a = 0
    if args.threshold_random_initial:
        threshold = np.random.rand(env.diag_size)
    else:
        threshold = args.threshold * np.ones(env.diag_size)

    step_idx = 0
    best_epoch_train_accuracy = 0
    for epoch in range(args.EPOCHS):
        env.reset()
        agent.train()
        num_batches = env.sample_size // args.batch_size
        steps_on_ave = 0
        pos_on_ave = 0
        accu_on_ave = 0
        for batch_idx in tqdm(range(num_batches), total=num_batches, desc=f"epoch {epoch}: "):
            step_idx += 1
            states = []
            action_m = []
            rewards_s = []
            action_s = []
            true_d = []
            true_diff_ind = []
            true_diff_prob = []

            s, true_disease, true_diff_indices, true_diff_probas, _ = env.initialize_state(args.batch_size)
            s_init = copy.deepcopy(s)
            s_final = copy.deepcopy(s)

            a_d, p_d = agent.choose_diagnosis(s)
            init_ent = entropy(p_d, axis = 1)
            
            done = (init_ent < threshold[a_d])
            right_diag = (a_d == env.disease) & done

            diag_ent = np.zeros(args.batch_size)
            finl_diag = np.zeros(args.batch_size).astype(int) - 1
            diag_ent[right_diag] = init_ent[right_diag]
            ent = init_ent

            for i in range(args.MAXSTEP):
                a_s = agent.choose_action_s(s)
                
                s_, r_s, done, right_diag, final_idx, ent_, a_d_ = env.step(s, a_s, done, right_diag, agent, init_ent, threshold, ent)
                s_final[final_idx] = s_[final_idx]
                diag_ent[right_diag] = ent_[right_diag]
                finl_diag[right_diag] = a_d_[right_diag]
                # print(max(finl_diag[right_diag]))
                # print(max(a_d_[right_diag]))
                # print(finl_diag[right_diag])
                # print(a_d_[right_diag])
                # input()
                if i == (args.MAXSTEP - 1):
                    r_s[~done] += 1

                states.append(s)
                rewards_s.append(r_s)
                action_s.append(a_s)
                true_d.append(true_disease)
                true_diff_ind.append(true_diff_indices)
                true_diff_prob.append(true_diff_probas)
                
                s = s_
                ent = ent_
                
                if all(done):
                    break
            
            diag = np.sum(done)
            s_final[~done] = s_[~done]

            _, all_step, ave_reward_s = agent.create_batch(states, rewards_s, action_s, true_d, true_diff_ind, true_diff_prob)
            a_d, p_d = agent.choose_diagnosis(s)
            
            t_d = (a_d == env.disease) & (~done)
            diag_ent[t_d] = entropy(p_d[t_d], axis = 1)
            finl_diag[t_d] = a_d[t_d]
            # print(max(finl_diag))
            for idx, item in enumerate(finl_diag):
                if item >= 0 and abs(threshold[item] - diag_ent[idx]) > 0.01:
                    threshold[item] = (args.lamb * threshold[item] + (1-args.lamb) * diag_ent[idx])   #update the threshold

            agent.update_param_rl()
            agent.update_param_c()
            
            accuracy = (sum(right_diag)+sum(t_d))/(args.batch_size)
            best_a = np.max([best_a, accuracy])

            ave_pos = np.sum(env.inquired_symptoms * env.all_state) / max(1, all_step)
            ave_step = all_step / args.batch_size

            threshold_list.append(threshold)

            print("==Epoch:", epoch+1, '\tAve. Accu:', accuracy, '\tBest Accu:', best_a, '\tAve. Pos:', ave_pos)
            print('Threshold:', threshold[:5], '\tAve. Step:', ave_step, '\tAve. Reward Sym.:', ave_reward_s, '\n')
            
            steps_on_ave = batch_idx / (batch_idx + 1) * steps_on_ave + 1 / (batch_idx + 1) * ave_step
            pos_on_ave = batch_idx / (batch_idx + 1) * pos_on_ave + 1 / (batch_idx + 1) * ave_pos
            accu_on_ave = batch_idx / (batch_idx + 1) * accu_on_ave + 1 / (batch_idx + 1) * accuracy
            
            # wandb logging
            results_dict = {
                "accuracy/train": accuracy,
                "best_accuracy/train": best_a,
                "average_pos/train": ave_pos,
                "average_step/train": ave_step,
                "average_symptom_reward/train": ave_reward_s,
                "epoch": epoch,
                "batch": step_idx - 1,
            }
            log_metric_data(results_dict)

        # wandb logging
        best_epoch_train_accuracy = max(accu_on_ave, best_epoch_train_accuracy)
        results_dict = {
            "epoch_accuracy/train": accu_on_ave,
            "epoch_best_accuracy/train": best_epoch_train_accuracy,
            "epoch_average_pos/train": pos_on_ave,
            "epoch_average_step/train": steps_on_ave,
            "epoch": epoch,
        }
        print("==Epoch:", epoch+1, '\tAve. EpochAccu:', accu_on_ave, '\tBest EpochAccu:', best_epoch_train_accuracy, '\tAve. EpochPos:', pos_on_ave)
        print('EpochThreshold:', threshold[:5], '\tAve. EpochStep:', steps_on_ave, '\n')
        log_metric_data(results_dict)

        agent.save_model(args)
        info = str(args.dataset) + '_' + str(args.threshold) + '_' + str(args.mu) + '_' + str(args.nu) + '_' + str(args.trail)
        with open(f'{args.save_dir}/threshold_changing_curve_'+info+'.pkl', 'wb') as f:
            pickle.dump(threshold_list, f)

        if args.eval_on_train_epoch_end:
            val_result = test(agent=agent, threshold=threshold, epoch=epoch, env=eval_env, log_flag=False)
            val_accuracy = val_result["epoch_accuracy/validation"]
            if best_val_accuracy is None or val_accuracy > best_val_accuracy:
                best_val_accuracy = val_accuracy
                agent.save_model(args, prefix="best_")
                patience = args.patience
            else:
                patience -= 1
            val_result["epoch_best_accuracy/validation"] = best_val_accuracy
            log_metric_data(val_result)
            if patience == 0:
                break
        else:
            if epoch == args.EPOCHS - 1:
                val_result = test(agent=agent, threshold=threshold, log_flag=True)
                best_val_accuracy = val_result["epoch_accuracy/validation"]

    report_results([dict(name="dev_metric", type="objective", value=-float(best_val_accuracy))])


提供的代码似乎是策略梯度强化学习模型的主要训练循环的一部分。以下是代码作用的概述： 它初始化环境并生成用于训练的患者。 它使用指定的参数创建 Policy_Gradient_pair_model 类的实例。 它初始化一个阈值列表和一个变量来跟踪最佳精度。 它为每个时期进入一个循环并执行以下步骤： 重置环境。 使用 train() 方法训练代理。 根据样本量和批次大小计算批次数。 初始化变量以跟踪平均步数、平均位置和平均准确度。 为每个批次进入一个循环并执行以下步骤： 选择一个动作并在环境中执行一个步骤。 存储状态、动作、奖励和其他信息以供体验重播。 使用强化学习和监督学习更新来更新代理的参数。 根据诊断出的疾病和熵更新阈值。 打印纪元、平均准确度、最佳准确度、平均位置、阈值、平均步长和平均症状奖励。 计算并更新平均步数、平均位置和平均准确度。 使用 log_metric_data() 函数记录批级指标。 计算并记录 epoch 级别的指标，包括平均准确度、最佳 epoch 准确度、平均位置和平均步长。 保存模型并将阈值变化曲线存储在文件中。 可选地在每个纪元结束时在验证集上评估模型并记录验证指标。 如果验证准确性提高，则模型将保存为“best_”前缀。 如果耐心值达到 0，训练循环可以提前停止。 如果未设置 eval_on_train_epoch_end 标志，则在训练循环完成后在验证集上评估模型，并记录最佳验证精度。 最后，它将最佳验证准确性报告为负目标值以指示最小化。 请注意，代码引用了其他函数和类（environment()、Policy_Gradient_pair_model、entropy()、agent.create_batch()、env.step()、agent.choose_diagnosis()、agent.choose_action_s()、agent.update_param_rl() , agent.update_param_c(), agent.save_model(), test(), log_metric_data(), report_results()) 未包含在提供的代码片段中。要完全理解此代码的行为和功能，您需要查看这些函数和类的实现。

In [ ]:
if __name__ == '__main__':


    parser = argparse.ArgumentParser(description='Process Settings')
    parser.add_argument('--dataset', type=str, default = 'casande',
                        help='Name of the dataset')
    parser.add_argument('--seed', type=int, default = 42,
                        help='set a random seed')
    parser.add_argument('--threshold', type=float, default = 1,
                        help='set a initial threshold')
    parser.add_argument('--threshold_random_initial', action="store_true",
                        help='randomly initialize threshold')
    parser.add_argument('--batch_size', type=int, default = 200,
                        help='batch_size for each time onpolicy sample collection')
    parser.add_argument('--eval_batch_size', type=int, default = 0,
                        help='batch_size for each time onpolicy evaluation')
    parser.add_argument('--eval_on_train_epoch_end', action="store_true",
                        help='evaluate at the end of each epoch')
    parser.add_argument('--EPOCHS', type=int, default = 100,
                        help='training epochs')
    parser.add_argument('--MAXSTEP', type=int, default = 30,
                        help='max inquiring turns of each MAD round')
    parser.add_argument('--patience', type=int, default = 10,
                        help='patience')
    parser.add_argument('--nu', type=float, default = 2.5,
                        help='nu')
    parser.add_argument('--mu', type=float, default = 1,
                        help='mu')
    parser.add_argument('--lr', type=float, default = 1e-4,
                        help='learning rate')
    parser.add_argument('--gamma', type=float, default = 0.99,
                        help='reward discount rate')
    parser.add_argument('--train', action="store_true",
                        help='whether test on the exsit result model or train a new model')
    parser.add_argument('--trail', type=int, default = 1)
    parser.add_argument('--eval_epoch', type=int, default = None, help='the epoch to use for evaluation')
    parser.add_argument('--lamb', type=float, default = 0.99,
                        help='polyak factor for threshold adjusting')
    parser.add_argument('--exp_name', type=str, default='EfficientRL', help='Experience Name')
    parser.add_argument('--save_dir', type=str, default='./output', help='directory to save the results')
    parser.add_argument('--checkpoint_dir', type=str, help='directory containing the checkpoints to restore')
    parser.add_argument('--train_data_path', type=str, required=True, help='path to the training data file')
    parser.add_argument('--val_data_path', type=str, required=True, help='path to the validation data file')
    parser.add_argument('--evi_meta_path', type=str, required=True, help='path to the evidences (symptoms) meta data',
                        default = './release_evidences.json'
    )
    parser.add_argument('--patho_meta_path', type=str, required=True, help='path to the pathologies (diseases) meta data',
                        default = './release_conditions.json'
    )
    parser.add_argument('--include_turns_in_state', action="store_true", help='whether to include turns on state')
    parser.add_argument('--date_time_suffix', action="store_true", help='whether to add time stamp suffix on the specified save_dir forlder')
    parser.add_argument('--no_differential', action="store_true", help='whether to not use differential')
    parser.add_argument('--no_initial_evidence', action="store_true", help='whether to not use the given initial evidence but randomly select one')
    parser.add_argument('--compute_eval_metrics', action="store_true", help='whether to compute custom evaluation metrics')
    parser.add_argument('--deterministic', action="store_true", help='deterministic evaluation')
    parser.add_argument('--prefix', type=str, default='', help='prefix to be added to the saved metric file.')
    parser.add_argument('--model_prefix', type=str, default='', help='prefix to be added to the model to be loaded.')
    
    args = parser.parse_args()
    if args.eval_batch_size == 0:
        args.eval_batch_size = args.batch_size
    if args.eval_epoch is None:
        args.eval_epoch = -1
    
    seed = args.seed
    np.random.seed(seed)
    torch.manual_seed(seed)
    random.seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    # setup wandb
    time_stamp = time.strftime("%m-%d-%H-%M-%S", time.localtime())
    if args.train:
        # add save_dir
        if args.date_time_suffix:
            args.save_dir = os.path.join(args.save_dir, time_stamp)
        os.makedirs(args.save_dir)

    init_logging(time_stamp, args)
    
    if args.train:        
        main()
    else:
        eval_metrics = test()
        write_json(eval_metrics, f"{args.checkpoint_dir}/EffRlMetrics_{args.dataset.lower()}{args.prefix}.json")


您提供的代码片段似乎是脚本的主要入口点。这是它的作用的细分： 它导入必要的模块并使用 argparse.ArgumentParser() 定义命令行参数。 命令行参数使用 parser.add_argument() 定义，用于与数据集、种子、阈值、批量大小、训练时期、学习率等相关的各种设置。 使用 parser.parse_args() 解析命令行参数并存储在 args 对象中。 一些额外的配置是根据解析的参数执行的，例如如果未明确设置，则将评估批量大小设置为等于训练批量大小。 使用 np.random.seed()、torch.manual_seed()、random.seed()、torch.cuda.manual_seed() 和 torch.cuda.manual_seed_all() 设置随机种子以实现可重复性。 如果 args.train 为 True，则脚本通过调用 main() 函数继续进行训练。否则，它将通过调用 test() 函数继续进行评估。 如果执行评估，则评估指标存储在 eval_metrics 变量中，并使用 write_json() 将指标写入 JSON 文件。 请注意，init_logging() 和 write_json() 等函数未包含在代码片段中，因此它们的功能并不明显。

main.py 脚本似乎是在医学证据收集环境中为基于策略梯度的代理运行训练循环的主要入口点。以下是脚本功能的细分： 导入必要的模块和库。 检查用于记录实验的可选库（WandB 和 MLflow）的可用性。 定义用于记录实验数据的函数。 定义 main() 函数，它封装了训练循环。 初始化环境并生成患者。 创建 Policy_Gradient_pair_model 代理的实例。 初始化诊断阈值。 遍历时代： 重置环境。 为每批数据训练代理。 更新阈值。 记录训练指标（准确性、位置、步骤、奖励等）。 保存代理的模型和阈值。 可选择在验证集和日志验证指标上评估代理。 将最佳验证准确性报告为客观指标。 此外，还定义了一个 test() 函数，用于在验证集上评估经过训练的代理。它初始化环境，加载代理的模型，并对验证数据运行推理。如果指定，它还会记录评估指标并计算其他指标。 请注意，代码的某些部分（例如环境、Policy_Gradient_pair_model 和 compute_metrics 函数）未包含在内，因此它们的详细信息和功能不可用。